# Moscow Housing Prices 

In this notebook we will solve the Moscow Housing problem using the standard Pipeline.

In [12]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Introduction and Overview

This challenge is about predicting the house prices of apartments in Moscow.
The dataset has information on over 33.000 apartments in Moscow.
We have 7 files to work with:
- **apartments_meta.json:** This file contains more information about the specific features for apartments we find in the test and training files.
- **buildings_meta.json:** Does the same as apartments_meta.json but for buildings.
- **apartments_train.csv:** Contains training data where each column is a feature and each row is specific to an apartment.
- **buildings_train.csv:** Same as apartments_train.csv but this file is for the buildings the apartments are located in. Important to notice that we can use the id from this file to merge with the building id from apartment_train.csv.
- **apartments_test.csv:** Almost the same as the training file but here our model has to make a column with predicted prices.
- **buildings_test.csv:** Contains information on the building that contain the apartments from the test set. 
- **moscow.png**: Map of Moscow. 

## 1. Data Exploration, Cleaning and Engineering (EDA)

In [15]:
%autoreload
 
import json
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
sns.set_style('darkgrid')
pd.set_option('display.max_colwidth', None)

In [119]:
apartments_train = pd.read_csv("Data/apartments_train.csv")
print("Loaded",len(apartments_train),"apartments from training set.")
apartments_train.head(3)

Loaded 23285 apartments from training set.


,id,seller,price,area_total,area_kitchen,area_living,floor,rooms,layout,ceiling,bathrooms_shared,bathrooms_private,windows_court,windows_street,balconies,loggias,condition,phones,building_id
0,0,3.0,7139520.0,59.2,12.5,31.0,2.0,2.0,NaN,2.65,0.0,2.0,0.0,1.0,NaN,NaN,NaN,1.0,4076
1,1,NaN,10500000.0,88.0,14.2,48.0,18.0,3.0,1.0,NaN,2.0,0.0,1.0,1.0,1.0,0.0,3.0,1.0,1893
2,2,3.0,9019650.0,78.5,22.5,40.8,12.0,3.0,NaN,2.65,0.0,2.0,1.0,1.0,NaN,NaN,NaN,1.0,5176


In [118]:
buildings_train = pd.read_csv("Data/buildings_train.csv")
print("Loaded",len(buildings_train),"buildings from training set.")
buildings_train.head(3)

Loaded 6791 buildings from training set.


,id,new,latitude,longitude,district,street,address,constructed,material,stories,elevator_without,elevator_passenger,elevator_service,parking,garbage_chute,heating
0,264,0.0,55.518117,37.355139,11.0,улица 1-я Нововатутинская,3,2019.0,2.0,12.0,1.0,1.0,1.0,0.0,1.0,3.0
1,6079,0.0,55.641920,37.352156,7.0,мкр. 10-й,к1014,1981.0,3.0,22.0,0.0,1.0,1.0,NaN,1.0,0.0
2,5829,0.0,55.770011,37.386220,7.0,Рублевское шоссе,68,2002.0,3.0,24.0,0.0,1.0,1.0,NaN,NaN,0.0


Below we can information about these features for apartments and buildings.

In [111]:
def describe_column(meta):
    """
    Utility function for describing a dataset column (see below for usage)
    """
    def f(x):
        d = pd.Series(name=x.name, dtype=object)
        m = next(m for m in meta if m['name'] == x.name)
        d['Type'] = m['type']
        d['#NaN'] = x.isna().sum()
        d['Description'] = m['desc']
        if m['type'] == 'categorical':
            counts = x.dropna().map(dict(enumerate(m['cats']))).value_counts().sort_index()
            d['Statistics'] = ', '.join(f'{c}({n})' for c, n in counts.items())
        elif m['type'] == 'real' or m['type'] == 'integer':
            stats = x.dropna().agg(['mean', 'std', 'min', 'max'])
            d['Statistics'] = ', '.join(f'{s}={v :.1f}' for s, v in stats.items())
        elif m['type'] == 'boolean':
            counts = x.dropna().astype(bool).value_counts().sort_index()
            d['Statistics'] = ', '.join(f'{c}({n})' for c, n in counts.items())
        else:
            d['Statistics'] = f'#unique={x.nunique()}'
        return d
    return f

def describe_data(data, meta):
    desc = data.apply(describe_column(meta)).T
    desc = desc.style.set_properties(**{'text-align': 'left'})
    desc = desc.set_table_styles([ dict(selector='th', props=[('text-align', 'left')])])
    return desc 

In [110]:
with open('data/apartments_meta.json') as f: 
    apartments_meta = json.load(f)
describe_data(apartments_train, apartments_meta)

,Type,#NaN,Description,Statistics
id,integer_id,0,Unique ID for apartment,#unique=23285
seller,categorical,8830,The type of apartment seller,"Agents(1895), Company(4768), Developer(6185), Owner(1607)"
price,real,0,The listed price of the apartment (TARGET),"mean=23556173.5, std=52643927.8, min=900000.0, max=2600000000.0"
area_total,real,0,Total area of the apartment,"mean=74.5, std=58.7, min=9.3, max=2181.0"
area_kitchen,real,4721,Total kitchen area in the apartment,"mean=12.6, std=6.6, min=1.0, max=100.0"
area_living,real,3882,Total living space area in the apartment,"mean=38.7, std=31.4, min=0.0, max=900.0"
floor,integer,0,Primary building floor of the apartment,"mean=9.0, std=8.3, min=1.0, max=95.0"
rooms,integer,0,Number of rooms in the apartment,"mean=2.2, std=1.1, min=1.0, max=6.0"
layout,categorical,17642,Overal apartment layout,"Adjacent(241), Adjacent_isolated(465), Isolated(4937)"
ceiling,real,11093,Ceiling height in the apartment,"mean=3.3, std=10.8, min=0.0, max=340.0"


In [115]:
with open('data/buildings_meta.json') as f: 
    buildings_meta = json.load(f)
describe_data(buildings_train, buildings_meta)

,Type,#NaN,Description,Statistics
id,integer_id,0,Unique ID of building for joining with apartments,#unique=6791
new,boolean,227,Whether it is an old or new building,"False(6141), True(423)"
latitude,real,0,Latitude coordinate of building,"mean=55.7, std=0.1, min=55.2, max=56.0"
longitude,real,0,Longitude coordinate of building,"mean=37.6, std=0.2, min=36.9, max=38.0"
district,categorical,1,Administriative district within Moscow,"Central(637), East(956), North(593), North-East(630), North-West(553), Novomoskovsk(344), South(758), South-East(672), South-West(900), Troitsk(146), West(535), Zelenograd(66)"
street,string,0,Bulding street name,#unique=1682
address,string,0,Building address (within street),#unique=1851
constructed,integer,479,Year when the building was constructed,"mean=1985.8, std=24.3, min=1855.0, max=2023.0"
material,categorical,1165,Primary building material used in building,"Block(679), Bricks(1226), Monolith(1312), Monolithic_brick(19), Panel(2384), Stalin_project(2), Wood(4)"
stories,integer,0,Total number of floors in the building,"mean=13.0, std=7.0, min=1.0, max=95.0"


It is very **important** to notice that features that have the type **categorical, are represented with numerical representation in the dataframes.** The first item as represented in the json file is 1.0 and the next is 2.0 and so on.

Since the field building id from the apartments data corresponds to the id field in the building data, we can merge these dataframes togheter so that we have only have a big dataframe to work with. This makes it easier to use different python tools and models.
But we have to check if all apartments have a building_id and also check that this is true for the test set.

In [48]:
apartments_test = pd.read_csv("Data/apartments_test.csv")
buildings_test = pd.read_csv("Data/buildings_test.csv")
print("Loaded",len(apartments_test),"apartments from test set.")
print("Loaded",len(buildings_test),"buildings from test set.")
print("How many missing values in building_id column from training set:",apartments_train.building_id.isnull().sum())
print("How many missing values in building_id column from test set:",apartments_test.building_id.isnull().sum())

Loaded 9937 apartments from test set.
Loaded 2931 buildings from test set.
How many missing values in building_id column from training set: 0
How many missing values in building_id column from test set: 0


We see that we have no missing values, now can merge the tables with a left join.

In [127]:
train_data = pd.merge(apartments_train, buildings_train.set_index('id'), how='left', left_on ='building_id', right_index = True)
pd.set_option('display.max_columns', None)
print("Number of rows in merged dataset",len(train_data))
print("Number of rows in the original apartments training set",len(apartments_train))
train_data.head(3)

Number of rows in merged dataset 23285
Number of rows in the original apartments training set 23285


,id,seller,price,area_total,area_kitchen,area_living,floor,rooms,layout,ceiling,bathrooms_shared,bathrooms_private,windows_court,windows_street,balconies,loggias,condition,phones,building_id,new,latitude,longitude,district,street,address,constructed,material,stories,elevator_without,elevator_passenger,elevator_service,parking,garbage_chute,heating
0,0,3.0,7139520.0,59.2,12.5,31.0,2.0,2.0,NaN,2.65,0.0,2.0,0.0,1.0,NaN,NaN,NaN,1.0,4076,1.0,55.544046,37.478055,11.0,Бунинские Луга ЖК,к2.5/2,2021.0,3.0,9.0,0.0,1.0,1.0,1.0,NaN,NaN
1,1,NaN,10500000.0,88.0,14.2,48.0,18.0,3.0,1.0,NaN,2.0,0.0,1.0,1.0,1.0,0.0,3.0,1.0,1893,0.0,55.861282,37.666647,2.0,улица Радужная,14к3,2010.0,3.0,25.0,0.0,1.0,1.0,1.0,NaN,0.0
2,2,3.0,9019650.0,78.5,22.5,40.8,12.0,3.0,NaN,2.65,0.0,2.0,1.0,1.0,NaN,NaN,NaN,1.0,5176,1.0,55.663299,37.515335,6.0,Саларьево Парк ЖК,38,2021.0,3.0,15.0,0.0,1.0,1.0,1.0,NaN,NaN


We have seen previously that we have missing values.
Let get a better view with a table showing the missing values for the training and the test data.

In [207]:
test_data = pd.merge(apartments_test, buildings_test.set_index('id'), how='left', left_on ='building_id', right_index = True)
print("Total Rows in test_data:",len(test_data))
test_data_nulls = test_data.isnull().sum()
train_data_nulls = train_data.isnull().sum()
test_data_nulls_df = test_data_nulls.to_frame().rename(columns={0:'Missing values test data'})
train_data_nulls_df = train_data_nulls.to_frame().rename(columns={0:'Missing values train data'})
pd.concat([train_data_nulls_df,test_data_nulls_df], axis=1)

Total Rows in test_data: 9937


,Missing values train data,Missing values test data
id,0,0.0
seller,8830,3942.0
price,0,NaN
area_total,0,0.0
area_kitchen,4721,2233.0
area_living,3882,2019.0
floor,0,0.0
rooms,0,0.0
layout,17642,7368.0
ceiling,11093,5018.0


We can see that in both the test data and training data, we have a lot of missing values.

## 2. Choosing an algorithm

## 3. Hyperparameter Optimization

## 4. Results Interpretation